In [1]:
import nltk
from nltk.tokenize import RegexpTokenizer
from collections import Counter, defaultdict, OrderedDict
import re
import sys
import os
import math
import string
import time
import operator
from itertools import islice
from functools import reduce
nltk.download('stopwords')

import pickle

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dongj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
book_content_w_ch =[]
with open('kjv.txt') as f:
    next(f)
    for line in f:
        book_content_w_ch.append(line)

In [3]:
book_content =[]
for i in book_content_w_ch:
    book_content.append(i.replace(i.split(' ')[0], ''))

In [4]:
def normalization(word):
    word= word.lower()
    word = word.replace('\n','')
    word  = re.sub(r'[^\w\s]', '', word)
    word = re.sub('[0-9]', '', word)
    return word

In [5]:
def preprocess(data):
    result = []
    for line in data:
        word = normalization(line)
        word = word.lower().strip().split()
        stopwords = nltk.corpus.stopwords.words("english")
        word = [w for w in word if not w in stopwords]
        word = " ".join(word)
        result.append(word)
    return result

In [6]:
with open('pickle_set/book_norm_text.pickle', 'rb') as f:
    book_normalized_text = pickle.load(f)

with open('pickle_set/book_collec_freq.pickle', 'rb') as f:
    book_collec_freq = pickle.load(f)

with open('pickle_set/book_doc_freq.pickle', 'rb') as f:
    book_doc_freq = pickle.load(f)
    
with open('pickle_set/book_term_freq.pickle', 'rb') as f:
    book_term_freq = pickle.load(f)

with open('pickle_set/book_dict_pos_output.pickle', 'rb') as f:
    book_dict_pos_output = pickle.load(f)

In [7]:
collection=[]
for i,j in book_collec_freq.items():
    collection.append(j)
collection_size = sum(collection)

In [8]:
collection_model =[]
for i in book_collec_freq.values():
    collection_model.append(i/collection_size)

In [9]:
q_doc=[]
for i,j in book_term_freq.items():
    q_mix=[]
    for k in book_collec_freq:
        if k not in j:
            q_mix.append(0)
        else:
            q_mix.append(j[k]/sum(book_term_freq[i].values()))
    q_doc.append(q_mix)

In [10]:
lambda_val = 0.4

In [11]:
lamb_tz =[]
for i in q_doc:
    tz =[]
    for j in range(len(i)):
        tz.append(lambda_val*i[j] + (1-lambda_val)*collection_model[j])
    lamb_tz.append(tz)

In [12]:
query= ['Jesus saves us']

In [13]:
query_content = preprocess(query)

In [14]:
query_mix = [0]*len(book_collec_freq.keys())

In [15]:
for i in range(len(book_collec_freq.keys())):
    for j in query_content[0].split():
        if list(book_collec_freq.keys())[i] == j:
            query_mix[i]+=1

In [16]:
mixture_doc = []
for i in lamb_tz:
    mixture = []
    for a,b in zip(i,query_mix):
        if b != 0:
            mixture.append(a**b)
    mixture_doc.append(mixture)

In [17]:
result =[]
for i in range(len(mixture_doc)):
    result.append([i, reduce(lambda x, y: x*y, mixture_doc[i])])

In [18]:
result.sort(key=lambda x: x[1], reverse=True)

In [23]:
print(query_content)

['jesus saves us']


In [22]:
for i in result[:10]:
    print(book_content_w_ch[i[0]])

Luke9:50 And Jesus said unto him, Forbid him not: for he that is not against us is for us.

2Cor4:14 Knowing that he which raised up the Lord Jesus shall raise up us also by Jesus, and shall present us with you.

Titus3:6 Which he shed on us abundantly through Jesus Christ our Saviour;

Luke17:13 And they lifted up their voices, and said, Jesus, Master, have mercy on us.

Acts1:21 Wherefore of these men which have companied with us all the time that the Lord Jesus went in and out among us,

1Cor15:57 But thanks be to God, which giveth us the victory through our Lord Jesus Christ.

2Cor5:18 And all things are of God, who hath reconciled us to himself by Jesus Christ, and hath given to us the ministry of reconciliation;

Eph2:6 And hath raised us up together, and made us sit together in heavenly places in Christ Jesus:

Rom4:24 But for us also, to whom it shall be imputed, if we believe on him that raised up Jesus our Lord from the dead;

Rom6:3 Know ye not, that so many of us as were ba